<img width="8%" alt="ImagineAPI.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/ImagineAPI.png" style="border-radius: 15%">

# ImagineAPI - Generate and Display Image
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=ImagineAPI+-+Generate+and+Display+Image:+Error+short+description">Bug report</a>

**Tags:** #imagineapi #image #generate #display #api #python #midjourney

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-11-13 (Created: 2023-10-31)

**Description:** This notebook will show how to create an image from a prompt using ImagineAPI.

**References:**
- [ImagineAPI Documentation](https://docs.imagineapi.dev/aypeeeye/image)

## Input

### Import libraries

In [ ]:
import naas
import requests
import pydash as py
import time
from IPython.display import Image, display

### Setup variables
- `api_key`: API key provided by ImagineAPI.dev. [Get your API key](https://docs.imagineapi.dev/aypeeeye/authentication)
- `imagineapi_url`: Your ImagineAPI URL
- `prompt`: Prompt to generate image

In [ ]:
api_key = naas.secret.get("IMAGINEAPI_API_KEY")
imagineapi_url = "https://xxxxx.imagineapi.dev/admin/login"
prompt = "Barack Obama funny profile picture"

## Model

### Create Image

In [ ]:
def create_image(
    api_key,
    base_url,
    prompt
):
    data = {
        "prompt": prompt,
    }
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    domain = base_url.split("https://")[1].split("/")[0]
    url  = f"https://{domain}/items/images/"
    res = requests.post(url, json=data, headers=headers)
    res.raise_for_status
    return py.get(res.json(), "data.id")

def get_image(
    api_key,
    base_url,
    image_id
):
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    domain = base_url.split("https://")[1].split("/")[0]
    url  = f"https://{domain}/items/images/{image_id}"
    res = requests.get(url, headers=headers)
    res.raise_for_status
    return res.json()

def generate_image(
    api_key,
    base_url,
    prompt
):
    # Init
    data = {}
    
    # Create Image
    image_id = create_image(
        api_key,
        base_url,
        prompt
    )
    
    while True:
        # Check status
        data = get_image(
            api_key,
            base_url,
            image_id
        )
        status = py.get(data, "data.status")
        progress = py.get(data, "data.progress")
        if status == "pending":
            progress = 0
        if status == "completed":
            progress = 100
        print(f"- Status: {status}")
        print(f"- Progress: {progress}%")
        print()
        if status in ["completed", "failed"]:
            break
        time.sleep(5)
    return data
    
image_data = generate_image(api_key, imagineapi_url, prompt)
image_data

## Output

### Display result

In [ ]:
urls = py.get(image_data, "data.upscaled_urls")
for url in urls:
    print("URL:", url)
    display(Image(url=url, width=200, height=200))